In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

import openai
import base64
import pandas as pd
from io import StringIO

import pandas as pd
import json
from datetime import datetime, timedelta

# Get the absolute path to the modules directory
module_path = os.path.abspath(os.path.join("..", "project_s"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from config import (
    DATA_WEEKLY_NOTES_PICKLE,
    DATA_SCREEN_TIME_PICKLE,
    DATA_CALENDAR_PICKLE,
    DATA_WEEKLY_PERCEPTION_PICKLE, 
    DATA_WEEKLY_NOTES_DIR,
    DATA_SCREEN_TIME_DIR,
    DATA_CALENDAR_DIR
)


## Perception

In [ ]:


from perception import perception_node

weekly_notes_df = pd.read_pickle(DATA_WEEKLY_NOTES_PICKLE)
screen_time_df = pd.read_pickle(DATA_SCREEN_TIME_PICKLE)
calendar_df = pd.read_pickle(DATA_CALENDAR_PICKLE)

perception_df = perception_node(weekly_notes_df, screen_time_df, calendar_df)

# Save result
perception_df.to_pickle(DATA_WEEKLY_PERCEPTION_PICKLE)
perception_df.head(2)



## Journal Archetypes

In [ ]:
from journal_archetypes import extract_journal_archetypes
from config import DATA_JOURNAL_PICKLE, OPENAI_API_KEY

journal_df = pd.read_pickle(DATA_JOURNAL_PICKLE)
themes = extract_journal_archetypes(journal_df, OPENAI_API_KEY)

state["journal_themes"] = themes


## Interpretation

In [ ]:
from config import DATA_WEEKLY_PERCEPTION_PICKLE, DATA_JOURNAL_PICKLE, OPENAI_API_KEY

from interpretation import interpretation_node

# Load data
weekly_df = pd.read_pickle(DATA_WEEKLY_PERCEPTION_PICKLE)
journal_df = pd.read_pickle(DATA_JOURNAL_PICKLE)

# Test single row
state = weekly_df.iloc[0].to_dict()
result = interpretation_node(state, journal_df, OPENAI_API_KEY)
print(result["interpretation"])




## Socratic

In [ ]:
from config import OPENAI_API_KEY, DATA_WEEKLY_PERCEPTION_JSON_FILE, DATA_JOURNAL_TXT_FILE

with open(DATA_JOURNAL_TXT_FILE, "r", encoding="utf-8") as file:
    journal_txt = file.read()

state = {
    "journal_text": journal_txt,
    "weekly_perception_text": pd.read_json(DATA_WEEKLY_PERCEPTION_JSON_FILE).iloc[1].to_dict()
}

In [ ]:
from project_s.project_s.socratic import socratic_node, format_reflection, deep_socratic_node
from config import OPENAI_API_KEY

updated_state = deep_socratic_node(state, OPENAI_API_KEY)
print(updated_state["socratic_observation"])



## Whisper

In [ ]:
from project_s.project_s.whisper import send_reflective_message
TWILIO_ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
TO_NUMBER = os.getenv('PHONE_NUMBER')
send_reflective_message(
    message=updated_state["socratic_observation"],
    account_sid=TWILIO_ACCOUNT_SID,
    auth_token=TWILIO_AUTH_TOKEN,
    from_number="whatsapp:+14155238886",
    to_number="whatsapp:"+TO_NUMBER
)

## Prepare data for graph states

In [ ]:
import pandas as pd
import os
import pandas as pd
import os

def split_weekly_notes_df(input_path: str, output_dir: str):
    """
    Loads a DataFrame of weekly notes and saves each row as a single-row DataFrame
    into its own pickle file using the 'week' column as the filename identifier.
    """
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_pickle(input_path)

    if "week" not in df.columns:
        raise ValueError("The dataframe must have a 'week' column.")

    for _, row in df.iterrows():
        week_label = row["week"]
        if pd.isna(week_label):
            continue

        safe_week = str(week_label).replace(" ", "_").replace("/", "-")
        output_path = os.path.join(output_dir, f"week_{safe_week}.pickle")

        # ✅ Save as single-row DataFrame
        pd.to_pickle(pd.DataFrame([row]), output_path)

    return f"Saved {len(df)} weekly entries to {output_dir}"




split_weekly_notes_df(DATA_WEEKLY_NOTES_PICKLE, DATA_WEEKLY_NOTES_DIR)



In [ ]:
import pandas as pd
import os

def split_screen_time_by_week(input_path: str, output_dir: str):
    """
    Splits a screen time DataFrame by 'week' and saves each group as a separate pickle file.

    Args:
        input_path (str): Path to the combined screen time pickle.
        output_dir (str): Directory to save weekly screen time pickle files.
    """
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_pickle(input_path)

    if "week" not in df.columns:
        raise ValueError("The dataframe must have a 'week' column.")

    for week, group in df.groupby("week"):
        safe_week = str(week).replace(" ", "_").replace("/", "-")
        output_path = os.path.join(output_dir, f"week_{safe_week}.pickle")
        pd.to_pickle(group, output_path)

    return f"Saved {len(df['week'].unique())} weekly screen time files to {output_dir}"

split_screen_time_by_week(DATA_SCREEN_TIME_PICKLE, DATA_SCREEN_TIME_DIR)    

In [ ]:
pd.read_pickle("../data/calendar/CT_calendar.pickle").head(2)

In [ ]:
import pandas as pd
import os

def split_calendar_by_week(input_path: str, output_dir: str):
    """
    Loads calendar data and splits it by week based on the 'start' datetime column.
    Saves each week's calendar as a separate pickle file.

    Args:
        input_path (str): Path to the combined calendar pickle.
        output_dir (str): Directory to save weekly pickle files.
    """
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_pickle(input_path)

    if "start" not in df.columns:
        raise ValueError("The dataframe must contain a 'start' datetime column.")

    # Ensure 'start' is datetime
    df["start"] = pd.to_datetime(df["start"])

    # Create a readable week label: e.g., "Jun 23–29"
    df["week"] = df["start"].dt.to_period("W").apply(
        lambda p: f"{p.start_time.strftime('%b %d')}–{(p.start_time + pd.Timedelta(days=6)).strftime('%d')}"
    )

    # Group and save
    for week, group in df.groupby("week"):
        safe_week = week.replace(" ", "_").replace("/", "-")
        output_path = os.path.join(output_dir, f"week_{safe_week}.pickle")
        pd.to_pickle(group, output_path)

    return f"Saved {len(df['week'].unique())} weekly calendar files to {output_dir}"

split_calendar_by_week(DATA_CALENDAR_PICKLE, DATA_CALENDAR_DIR)

